In [9]:
import os
import SimpleITK as sitk
import glob
import monai
from monai.transforms import (
    AsDiscrete,
    AsDiscreted,
    EnsureChannelFirstd,
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandCropByPosNegLabeld,
    SaveImaged,
    ScaleIntensityRanged,
    Spacingd,
    AddChanneld,
    SpatialPadd,
    RandRotate90d,
    RandShiftIntensityd,
    EnsureTyped,
    EnsureType,
    MapTransform,
    Resized,
    Invertd,
    ToTensord,
    NormalizeIntensityd,
    RandFlipd,
    Lambdad,
    Activations,
    AsDiscrete,
)
from monai.metrics import ROCAUCMetric
from monai.data import CacheDataset, ThreadDataLoader,DataLoader, Dataset, decollate_batch,load_decathlon_datalist
import torch

In [10]:
pretrained_path = 'best_metric_model_classification3d_dict.pth'
data_dir = './Mixed2/'

all_images = sorted(glob.glob(os.path.join(data_dir,  '*.nii.gz')))

# split = int(len(all_images) * 0.6)
# # train_image = all_images[:split]
# # val_image = all_images[split: ]
# train_image = all_images[split: ] 
# val_image = all_images[:split]

val_dict = [{'image': image_name, 'label': int(image_name.split('_')[-1].replace('.nii.gz', ''))} for image_name in all_images]
len(val_dict)


66

In [11]:
val_transforms = Compose(
        [
            LoadImaged(keys=["image", ]),
            EnsureChannelFirstd(keys=["image", ]),
            Orientationd(keys=["image", ], axcodes="RAS"),
            Resized(keys=["image"], spatial_size=(96, 96, 32)),
            NormalizeIntensityd(keys="image", nonzero=True, channel_wise=True),
            ToTensord(keys=['image', 'label'])
        ]
    )

val_ds = CacheDataset(data=val_dict, transform=val_transforms, cache_rate=1.0, num_workers=12)
# create a validation data loader
val_loader = DataLoader(val_ds, batch_size=1, num_workers=16, pin_memory=True)

Loading dataset: 100%|█████████████████████████████████████████████████████████████████| 66/66 [00:02<00:00, 32.76it/s]


In [2]:

device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
model = monai.networks.nets.resnet18(spatial_dims=3, n_input_channels=1, num_classes=2).to(device)
model.load_state_dict(torch.load(pretrained_path, map_location=torch.device('cuda:0')))

NameError: name 'torch' is not defined

In [13]:
post_transforms = Compose([Activations(softmax=True), 
                    AsDiscrete(argmax=True)
                    ])

model.eval()
results = []
with torch.no_grad():
    
    y = torch.tensor([], dtype=torch.long, device=device)
    for val_data in val_loader:
        val_images, val_labels = val_data["image"].to(device), val_data["label"].to(device)
       
        y_pred = model(val_images)   
        predict = post_transforms(decollate_batch(y_pred)[0])

#         print('Label | Prediction: ',val_labels.item(),'|',  int(predict.item()))
        result = int(val_labels.item() == int(predict.item()))
        results.append(result)
        if result == 0:
            print(val_data['image_meta_dict']['filename_or_obj'][0])
sum(results) / len(results)

Mixed2/2019_07_31_5344100_5934655_+C_Reg2_0000_0.nii.gz
Mixed2/2019_09_06_1298758_5966750_+C_Reg2_0000_1.nii.gz
Mixed2/2019_10_08_5430679_6200746_+C_Reg2_0000_0.nii.gz
Mixed2/2019_11_28_5518320_6177727_+C_Reg2_0000_0.nii.gz
Mixed2/2020_01_12_5563177_6260624_+C_Reg2_0000_0.nii.gz
Mixed2/2020_01_20_5577636_6216554_+C_Reg2_0000_0.nii.gz
Mixed2/2020_03_10_1364226_6235750_+C_Reg2_0000_0.nii.gz
Mixed2/2020_03_17_5614238_6417032_+C_Reg2_0000_0.nii.gz
Mixed2/2020_03_26_5622217_6311607_+C_Reg2_0000_0.nii.gz
Mixed2/2020_04_27_1723627_6771393_+C_Reg2_0000_0.nii.gz
Mixed2/2020_05_20_731908_6266136_+C_Reg2_0000_1.nii.gz
Mixed2/2020_07_20_3534085_6642320_+C_Reg2_0000_0.nii.gz
Mixed2/2020_07_25_3753963_6624869_+C_Reg2_0000_0.nii.gz


0.803030303030303

In [65]:
# 后40%
Mixed/2018_07_09_4778901_4966875_+C_Reg2_0000_1.nii.gz
Mixed/2018_08_01_4826541_5196840_+C_Reg2_0000_0.nii.gz
Mixed/2018_08_13_4827203_5050551_+C_Reg2_0000_0.nii.gz
Mixed/2018_10_30_4948949_5278769_+C_Reg2_0000_1.nii.gz
Mixed/2018_11_23_4981484_5448008_+C_Reg2_0000_0.nii.gz
Mixed/2018_12_07_4994670_5471998_+C_Reg2_0000_0.nii.gz
Mixed/2019_03_01_8037719_5528736_+C_Reg2_0000_0.nii.gz
Mixed/2019_03_26_5149647_5632190_+C_Reg2_0000_1.nii.gz
Mixed/2019_04_08_5165615_5651423_+C_Reg2_0000_0.nii.gz
Mixed/2019_04_12_1584882_5677318_+C_Reg2_0000_1.nii.gz
Mixed/2019_04_29_5197747_5520710_+C_Reg2_0000_1.nii.gz
Mixed/2019_06_13_15_5270810_5767438_+C_Reg2_0000_0.nii.gz
Mixed/2019_06_21_5283203_5920987_+C_Reg2_0000_1.nii.gz
Mixed/2019_07_30_11_5341296_5882035_+C_Reg2_0000_0.nii.gz

SyntaxError: invalid decimal literal (998485614.py, line 1)

In [53]:
# 前60%里的
Mixed/2014_04_11_3125784_2619932_+C_Reg2_0000_0.nii.gz
Mixed/2014_08_26_8049503_2780230_+C_Reg2_0000_1.nii.gz
Mixed/2014_09_10_3263542_2793327_+C_Reg2_0000_1.nii.gz
Mixed/2014_12_16_666388_2809931_+C_Reg2_0000_1.nii.gz
Mixed/2015_01_05_3351046_2927261_+C_Reg2_0000_0.nii.gz
Mixed/2017_02_09_21350_4175057_+C_Reg2_0000_1.nii.gz
Mixed/2017_06_19_3289508_4348412_+C_Reg2_0000_1.nii.gz
Mixed/2017_07_07_4272111_4317065_+C_Reg2_0000_1.nii.gz
Mixed/2017_07_25_4302448_4373232_+C_Reg2_0000_1.nii.gz
Mixed/2017_12_06_4498974_4710164_+C_Reg2_0000_1.nii.gz
Mixed/2018_03_29_4641984_4867060_+C_Reg2_0000_0.nii.gz
Mixed/2018_04_18_1088138_4892941_+C_Reg2_0000_1.nii.gz
Mixed/2018_04_27_4685497_5064354_+C_Reg2_0000_0.nii.gz

14.72